# EU Energy Map

**Version 1.2**

https://github.com/kuranez/EU-Energy-Map

# I. Import

In [ ]:
import os
import json

import panel as pn
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.colors import sample_colorscale

# II. Settings

In [ ]:
# Initialize Panel extension
pn.extension('tabulator', 'plotly', design='material', sizing_mode='stretch_width')

# Set MapBox access token
MAPBOX_TOKEN = 'pk.eyJ1Ijoia3VyYW5leiIsImEiOiJjbTJmMjI0d2kwNDVxMnFzYXNldnc1N2VsIn0.t11TYpF2QBdid-hQfW8mig'

# III. Functions

In [ ]:
@pn.cache
def iso2_to_flag(iso2_code):
    """Convert ISO2 country code to flag emoji."""
    return chr(0x1F1E6 + ord(iso2_code[0]) - ord('A')) + chr(0x1F1E6 + ord(iso2_code[1]) - ord('A'))

In [ ]:
def load_data():
    """Load energy data and European geojson data."""
    data = pd.read_csv('./data/nrg_ind_ren_linear.csv')
    europe_gdf = gpd.read_file("europe.geojson")
    
    merged_data = europe_gdf.merge(data, left_on='CNTR_ID', right_on='geo')
    
    merged_data.rename(columns={
        'nrg_bal': 'Energy Type', 'TIME_PERIOD': 'Year',
        'OBS_VALUE': 'Renewable Percentage', 'geo': 'Code',
        'NAME_ENGL': 'Country'
    }, inplace=True)
    
    energy_type_map = {
        'REN': 'Renewable Energy Total',
        'REN_ELC': 'Renewable Electricity',
        'REN_HEAT_CL': 'Renewable Heating and Cooling',
        'REN_TRA': 'Renewable Energy in Transport'
    }
    merged_data['Energy Type'] = merged_data['Energy Type'].replace(energy_type_map)
    
    merged_data.drop(columns=['LAST UPDATE', 'freq', 'unit', 'OBS_FLAG'], inplace=True)
    merged_data[['Year', 'Renewable Percentage']] = merged_data[['Year', 'Renewable Percentage']].apply(pd.to_numeric)
    merged_data['Renewable Percentage'] = merged_data['Renewable Percentage'].round(1)
    merged_data['Flag'] = merged_data['Code'].apply(iso2_to_flag)
    
    final_columns = [
        'Code', 'Flag', 'Country', 'Energy Type', 'Renewable Percentage', 'Year',
        'CNTR_ID', 'ISO3_CODE', 'geometry'
    ]
    return merged_data[final_columns]

In [ ]:
def filter_data(merged_data):
    """Filter and structure dataset for visualization."""
    eu_countries = {"AT", "BE", "BG", "HR", "CY", "CZ", "DK", "EE", "FI", "FR", "DE", "GR", "HU", "IE", "IT", "LV", "LT", 
                    "LU", "MT", "NL", "PL", "PT", "RO", "SK", "SI", "ES", "SE"}
    df_renewable = merged_data[(merged_data['Energy Type'] == 'Renewable Energy Total') & merged_data['Code'].isin(eu_countries)]
    df_eu_total = df_renewable.groupby('Year', as_index=False)['Renewable Percentage'].mean()
    return df_renewable, df_eu_total

In [ ]:
@pn.cache
def create_choropleth_map(df_year):
    """Generate a Choropleth map of renewable energy percentages."""
    fig = go.Figure(go.Choroplethmapbox(
        geojson=json.load(open('europe.geojson')),
        locations=df_year['Code'],
        z=df_year['Renewable Percentage'],
        colorscale="Viridis",
        marker_opacity=0.8, marker_line_width=0.5,
        featureidkey="properties.CNTR_ID"
    ))
    fig.update_layout(mapbox_accesstoken=MAPBOX_TOKEN, mapbox_style="carto-positron", mapbox_zoom=3,
                      mapbox_center={"lat": 54, "lon": 15}, margin={"r": 0, "t": 0, "l": 0, "b": 0})
    return fig

In [ ]:
@pn.cache
def create_bar_chart_year(df_year, year):
    """Generate a bar chart of renewable energy by country."""
    df_year = df_year.sort_values(by='Renewable Percentage')
    fig = px.bar(df_year, x='Country', y='Renewable Percentage', color='Renewable Percentage', color_continuous_scale='Viridis')
    fig.update_layout(title=f"Renewable Energy in {year}", yaxis_title="Renewable Energy (%)")
    return fig

In [ ]:
@pn.cache
def create_bar_chart_country(df_eu_total, df_country, country):
    """Generate a bar chart of a country's renewable energy progression."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_eu_total['Year'], y=df_eu_total['Renewable Percentage'], mode='lines+markers', name="EU Avg"))
    fig.add_trace(go.Bar(x=df_country['Year'], y=df_country['Renewable Percentage'], name=country))
    fig.update_layout(title=f"{country} Renewable Energy (2004-2022)", yaxis_title="Renewable Energy (%)")
    return fig

# IV.  Main

In [ ]:
# Load and preprocess data
merged_data = load_data()
df_renewable, df_eu_total = filter_data(merged_data)
selected_year, selected_country = 2022, "Germany"
df_year = df_renewable[df_renewable['Year'] == selected_year]
df_country = df_renewable[df_renewable['Country'] == selected_country]

# V. Dashboard

In [ ]:
# Widgets
year_slider = pn.widgets.IntSlider(name='Year', start=2004, end=2022, step=1, value=selected_year)
country_selection = pn.widgets.Select(name='Country', options=df_renewable['Country'].unique().tolist(), value=selected_country)

In [ ]:
# Interactive components
interactive_map = pn.bind(create_choropleth_map, year_slider)
interactive_bar_chart_year = pn.bind(create_bar_chart_year, year_slider)
interactive_bar_chart_country = pn.bind(create_bar_chart_country, df_eu_total, country_selection.param.value)

In [ ]:
# Layout
tabs = pn.Tabs(('Year Filter', pn.Column(year_slider, pn.pane.Plotly(interactive_bar_chart_year))),
               ('Country Filter', pn.Column(country_selection, pn.pane.Plotly(interactive_bar_chart_country))))
layout = pn.Column(pn.pane.Plotly(interactive_map), tabs)

In [ ]:
pn.template.FastListTemplate(title="EU Energy Map", sidebar=[], main=[layout]).servable()